In [52]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import gensim
from gensim import corpora

plt.rc('font', family='Malgun Gothic')
font_path = r"C:/windows/fonts/malgun.ttf"

In [3]:
# for colab
"""!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!cd Mecab-ko-for-Google-Colab
!bash /content/Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh
"""

'!pip install konlpy\n!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git\n!cd Mecab-ko-for-Google-Colab\n!bash /content/Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh\n'

In [4]:
from konlpy.tag import Mecab
tagger = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")

In [5]:
yymm = "2401"

file_path = '../data/'  # 파일 경로를 지정하세요
fn= f'epic_metadata_{yymm}.xlsx'

df = pd.read_excel(file_path+fn)

In [6]:
df.head()

,Unnamed: 0,자료명,발간일,발간처,요약
0,246696,2024년 2월 물가연동국고채 종목별 연동계수,2024.01.02,기획재정부 국고국 국채과,기획재정부는 1.2.(화) 2024년 2월 물가연동국고채 종목별 연동계수를 발표하였...
1,246697,「2024년부터 이렇게 달라집니다」 책자 발간,2023.12.28,기획재정부 기획조정실 혁신정책담당관,기획재정부는 12.31.(일) 2024년부터 달라지는 제도와 법규사항 등을 알기 쉽...
2,246698,국세물납증권 56개 종목 공개매각 실시,2024.01.02,기획재정부 국고국 출자관리과,정부는 국유재산정책심의위원회에서 의결한 「2023년도 제3차 국세물납증권 매각 예정...
3,246699,"조달청, 1월 대형사업 총 163건, 1조 8,901억 원 상당 입찰 예정",2024.01.02,조달청,"조달청은 1.2.(화) ’24년 1월 한 달 동안 총 163건 1조 8,901억 원..."
4,246700,인구감소지역 『생활인구』 시범산정 결과 공표,2024.01.01,통계청,통계청과 행정안전부은 1.1.(월) 7개의 인구감소지역에 대해 「생활인구」를 시범 ...


In [7]:
stop_words = "안 간 붙임 참고 참조 첨부 총리 장관 겸 청 실 는 은 가 등 원 조 개 이 저 다만 하지만 그러나 중 억 년 월 화 수 목 금 토 일 것 줄 만 건 또한 아울러 그리고 전년 금년 이번 올해 내년 기획 재정부 첨부 파일 내용 발표 위원회 감독원 부"
stop_words = stop_words.split(" ")

In [8]:
excluson = ["기획재정부","정부",'과학기술정보통신부','과기정통부','농림축산식품부','농식품부','금융위원회','금융위','금융감독원','금감원','산업통상자원부','산업부',
            '환경부','해양수산부','해수부','공정거래위원회','공정위','식약처','식품의약품안전처','고용노동부','고용부','국토교통부','국토부','중소벤처기업부','중기부',
            '통계청','국세청','관세청','조달청','특허청','통일부','보건복지부','복지부','교육부','한국은행',
            '계획임.','예정임.','밝혔다.','발표하였다.']

In [28]:
summary_list = df['요약'].tolist()

documents_list = []
for summary in summary_list:
    document = []
    for ex in excluson:
        summary = summary.replace(ex,'')
    for noun in tagger.nouns(summary):
        if noun not in stop_words:
            document.append(noun)
    documents_list.append(document)

In [30]:
documents_list[:2]

[['물가', '연동', '국고', '종목별', '연동', '계수'],
 ['제도',
  '법규',
  '사항',
  '정리',
  '책자',
  '발간',
  '책자',
  '기관',
  '정책',
  '분야',
  '시기',
  '기관',
  '구성',
  '주요',
  '이해',
  '삽화',
  '제시',
  '청년',
  '여성',
  '부모',
  '다문화가정',
  '신혼',
  '부부',
  '계층',
  '특화',
  '정책',
  '분야',
  '제시',
  '한편',
  '사회',
  '안전',
  '탄소',
  '중립',
  '교육',
  '지원',
  '관련',
  '제도',
  '변경',
  '다수',
  '포함',
  '책자',
  '초',
  '지방',
  '자치',
  '단체',
  '공공',
  '도서관',
  '점자',
  '도서관',
  '권',
  '배포',
  '비치',
  '온라인',
  '공개',
  '분야',
  '주요',
  '삽화']]

In [31]:
documents = [' '.join(words) for words in documents_list]

In [32]:
documents[:2]

['물가 연동 국고 종목별 연동 계수',
 '제도 법규 사항 정리 책자 발간 책자 기관 정책 분야 시기 기관 구성 주요 이해 삽화 제시 청년 여성 부모 다문화가정 신혼 부부 계층 특화 정책 분야 제시 한편 사회 안전 탄소 중립 교육 지원 관련 제도 변경 다수 포함 책자 초 지방 자치 단체 공공 도서관 점자 도서관 권 배포 비치 온라인 공개 분야 주요 삽화']

In [39]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(documents)

## lsa

In [49]:
svd_model = TruncatedSVD(n_components=30, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

30

In [50]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('지원', 0.27294), ('사업', 0.25721), ('기업', 0.19469), ('기술', 0.1725), ('개발', 0.13122)]
Topic 2: [('기술', 0.27699), ('사업', 0.27374), ('연구', 0.20763), ('개발', 0.2019), ('기업', 0.18924)]
Topic 3: [('금융', 0.29629), ('대출', 0.26472), ('대비', 0.21643), ('증가', 0.20062), ('수출', 0.18637)]
Topic 4: [('대비', 0.25099), ('증가', 0.21283), ('달러', 0.18695), ('감소', 0.18586), ('수출', 0.18538)]
Topic 5: [('금융', 0.31184), ('기술', 0.28208), ('대출', 0.21604), ('연구', 0.20494), ('개발', 0.16803)]
Topic 6: [('지역', 0.22646), ('연구', 0.1777), ('인구', 0.17129), ('지방', 0.15899), ('개발', 0.12751)]
Topic 7: [('점검', 0.13099), ('경제', 0.13069), ('간담회', 0.12288), ('공급', 0.11979), ('기업', 0.1152)]
Topic 8: [('의료', 0.47142), ('마약', 0.18012), ('창업', 0.1671), ('교육', 0.16033), ('기업', 0.1538)]
Topic 9: [('의료', 0.26207), ('지원', 0.17826), ('대출', 0.17298), ('연구', 0.16029), ('식품', 0.15855)]
Topic 10: [('수출', 0.40543), ('지원', 0.18598), ('의료', 0.14948), ('지역', 0.14566), ('달러', 0.11537)]
Topic 11: [('교육', 0.3041), ('재산', 0.20622), ('지식', 0.1

## lda

In [53]:
id2word=corpora.Dictionary(documents_list)
id2word.filter_extremes(no_below = 20) #20회 이하로 등장한 단어는 삭제
texts = documents_list
corpus=[id2word.doc2bow(text) for text in texts]

mallet_path = 'mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models..LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

AttributeError: module 'gensim.models' has no attribute 'wrappers'